# 020: The Wilson-Cowan Model

Modeling excitatory-inhibitory population dynamics and neural oscillations.

## Learning Objectives

By the end of this tutorial, you will be able to:

- Understand the Wilson-Cowan equations for E/I population dynamics
- Explain the sigmoidal transfer function and its biological interpretation
- Analyze bifurcation diagrams to identify oscillatory regimes
- Build whole-brain networks using structural connectivity and delays

## Prerequisites

- **Tutorial 000**: Getting Started with BrainMass
- **Tutorial 010**: Hopf Oscillator (for comparison of oscillation mechanisms)
- Basic understanding of differential equations
- Familiarity with excitatory/inhibitory neural populations

## Background / Theory

### The Wilson-Cowan Model

The **Wilson-Cowan model** (1972) is a foundational neural mass model describing the interaction between excitatory (E) and inhibitory (I) neural populations. It was one of the first models to capture emergent oscillatory dynamics from E/I balance.

### The Equations

$$
\begin{aligned}
\tau_E \frac{dr_E}{dt} &= -r_E + (1 - r \cdot r_E) \cdot F_E(w_{EE} r_E - w_{EI} r_I + I_E) \\
\tau_I \frac{dr_I}{dt} &= -r_I + (1 - r \cdot r_I) \cdot F_I(w_{IE} r_E - w_{II} r_I + I_I)
\end{aligned}
$$

where:
- $r_E, r_I$: Excitatory and inhibitory population activities
- $\tau_E, \tau_I$: Time constants
- $w_{XY}$: Connection weights from population Y to X
- $r$: Refractory parameter (limits maximum firing)
- $I_E, I_I$: External inputs

### The Sigmoid Transfer Function

$$
F(x; a, \theta) = \frac{1}{1 + e^{-a(x - \theta)}} - \frac{1}{1 + e^{a\theta}}
$$

| Parameter | Role |
|-----------|------|
| $a$ | Gain (steepness of response) |
| $\theta$ | Threshold (input level for half-activation) |

### Connection Weights

| Weight | Meaning | Typical Effect |
|--------|---------|----------------|
| $w_{EE}$ | E→E recurrence | Positive feedback, bistability |
| $w_{EI}$ | I→E inhibition | Negative feedback, stabilization |
| $w_{IE}$ | E→I excitation | Drives inhibition |
| $w_{II}$ | I→I recurrence | Self-inhibition |

### Dynamical Regimes

The Wilson-Cowan model exhibits rich dynamics:

1. **Stable fixed points**: E/I balance at rest
2. **Limit cycle oscillations**: Rhythmic E/I alternation
3. **Bistability**: Multiple stable states
4. **Excitable dynamics**: Transient responses to perturbations

## Implementation

### Step 1: Setup and Imports

In [ ]:
import brainmass
import brainstate
import braintools
import brainunit as u
import matplotlib.pyplot as plt
import numpy as np

# Set simulation time step
brainstate.environ.set(dt=0.1 * u.ms)

### Step 2: Single Node Simulation

Create a Wilson-Cowan oscillator with noise on both E and I populations:

In [ ]:
# Create Wilson-Cowan node with OU noise
node = brainmass.WilsonCowanStep(
    1,  # single node
    noise_E=brainmass.OUProcess(1, sigma=0.01, tau=5.0 * u.ms),
    noise_I=brainmass.OUProcess(1, sigma=0.01, tau=5.0 * u.ms),
)

# Initialize states
brainstate.nn.init_all_states(node)

# Print default parameters
print("Default Wilson-Cowan parameters:")
print(f"  wEE = {node.wEE.value()}, wEI = {node.wEI.value()}")
print(f"  wIE = {node.wIE.value()}, wII = {node.wII.value()}")
print(f"  tau_E = {node.tau_E.value()}, tau_I = {node.tau_I.value()}")

In [ ]:
# Define simulation step
def step_run(i):
    with brainstate.environ.context(i=i, t=i * brainstate.environ.get_dt()):
        node.update(rE_inp=0.1)  # constant excitatory drive
        return node.rE.value, node.rI.value

# Run simulation (1000 ms)
indices = np.arange(10000)
rE_trace, rI_trace = brainstate.transform.for_loop(step_run, indices)

### Step 3: Visualize E/I Dynamics

In [ ]:
t_ms = indices * brainstate.environ.get_dt()

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Time series
axes[0].plot(t_ms, rE_trace[:, 0], 'b-', label='E (excitatory)', linewidth=1)
axes[0].plot(t_ms, rI_trace[:, 0], 'r-', label='I (inhibitory)', linewidth=1, alpha=0.8)
axes[0].set_xlabel('Time (ms)')
axes[0].set_ylabel('Activity')
axes[0].set_title('E/I Population Dynamics')
axes[0].legend()
axes[0].set_xlim([0, 500])

# Phase portrait
axes[1].plot(rE_trace[:, 0], rI_trace[:, 0], 'k-', linewidth=0.3, alpha=0.7)
axes[1].plot(rE_trace[0, 0], rI_trace[0, 0], 'go', markersize=10, label='Start')
axes[1].plot(rE_trace[-1, 0], rI_trace[-1, 0], 'r*', markersize=12, label='End')
axes[1].set_xlabel('E activity')
axes[1].set_ylabel('I activity')
axes[1].set_title('E-I Phase Portrait')
axes[1].legend()

# Power spectrum
from scipy import signal
fs = 1000 / float(brainstate.environ.get_dt() / u.ms)  # sampling frequency
f, Pxx = signal.welch(rE_trace[2000:, 0], fs=fs, nperseg=2048)
axes[2].semilogy(f, Pxx)
axes[2].set_xlabel('Frequency (Hz)')
axes[2].set_ylabel('Power')
axes[2].set_title('Power Spectrum (E population)')
axes[2].set_xlim([0, 200])

plt.tight_layout()
plt.show()

### Step 4: Bifurcation Analysis

Sweep external input to reveal oscillatory and fixed-point regimes:

In [ ]:
# Input values to sweep
exc_inputs = np.arange(0, 5.5, 0.05)

# Create nodes for all input values simultaneously
nodes = brainmass.WilsonCowanStep(exc_inputs.size)
brainstate.nn.init_all_states(nodes)

def step_sweep(i):
    with brainstate.environ.context(i=i, t=i * brainstate.environ.get_dt()):
        return nodes.update(rE_inp=exc_inputs)

# Run simulation
indices = np.arange(15000)
rE_sweep = brainstate.transform.for_loop(step_sweep, indices)

# Discard transient and compute min/max
rE_steady = rE_sweep[5000:, :]
max_exc = rE_steady.max(axis=0)
min_exc = rE_steady.min(axis=0)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Bifurcation diagram
axes[0].fill_between(exc_inputs, min_exc, max_exc, alpha=0.3, color='blue', label='Oscillation range')
axes[0].plot(exc_inputs, max_exc, 'b-', linewidth=2, label='Max rE')
axes[0].plot(exc_inputs, min_exc, 'b-', linewidth=2, label='Min rE')
axes[0].set_xlabel('External Input (I_E)')
axes[0].set_ylabel('E activity (min/max)')
axes[0].set_title('Bifurcation Diagram')
axes[0].legend()

# Identify regimes
oscillation_amplitude = max_exc - min_exc
axes[1].plot(exc_inputs, oscillation_amplitude, 'k-', linewidth=2)
axes[1].axhline(0.01, color='r', linestyle='--', label='Oscillation threshold')
axes[1].fill_between(exc_inputs, 0, oscillation_amplitude, 
                     where=oscillation_amplitude > 0.01, alpha=0.3, color='green', label='Oscillatory')
axes[1].fill_between(exc_inputs, 0, oscillation_amplitude, 
                     where=oscillation_amplitude <= 0.01, alpha=0.3, color='red', label='Fixed point')
axes[1].set_xlabel('External Input (I_E)')
axes[1].set_ylabel('Oscillation Amplitude')
axes[1].set_title('Dynamical Regime Classification')
axes[1].legend()

plt.tight_layout()
plt.show()

# Find oscillation boundaries
osc_threshold = 0.01
osc_indices = np.where(oscillation_amplitude > osc_threshold)[0]
if len(osc_indices) > 0:
    print(f"Oscillatory regime: I_E in [{exc_inputs[osc_indices[0]]:.2f}, {exc_inputs[osc_indices[-1]]:.2f}]")

### Step 5: Nullcline Analysis

Understanding the fixed points through nullcline intersection:

In [ ]:
# Sigmoid function
def F(x, a, theta):
    return 1 / (1 + np.exp(-a * (x - theta))) - 1 / (1 + np.exp(a * theta))

# Parameters (default values)
wEE, wEI, wIE, wII = 12., 13., 4., 11.
a_E, theta_E = 1.2, 2.8
a_I, theta_I = 1.0, 4.0
r = 1.0
I_E = 0.5  # external input

# Create grid
rE_range = np.linspace(0, 1, 200)
rI_range = np.linspace(0, 1, 200)
RE, RI = np.meshgrid(rE_range, rI_range)

# E-nullcline: drE/dt = 0
# rE = (1 - r*rE) * F(wEE*rE - wEI*rI + I_E)
# Solve numerically by finding where drE/dt changes sign
drE = -RE + (1 - r * RE) * F(wEE * RE - wEI * RI + I_E, a_E, theta_E)
drI = -RI + (1 - r * RI) * F(wIE * RE - wII * RI, a_I, theta_I)

fig, ax = plt.subplots(1, 1, figsize=(8, 6))

# Plot nullclines
ax.contour(RE, RI, drE, levels=[0], colors=['blue'], linewidths=2)
ax.contour(RE, RI, drI, levels=[0], colors=['red'], linewidths=2)

# Vector field
skip = 10
ax.quiver(RE[::skip, ::skip], RI[::skip, ::skip], 
          drE[::skip, ::skip], drI[::skip, ::skip], alpha=0.5)

# Overlay trajectory
ax.plot(rE_trace[:, 0], rI_trace[:, 0], 'k-', linewidth=0.5, alpha=0.7, label='Trajectory')

ax.set_xlabel('E activity (rE)')
ax.set_ylabel('I activity (rI)')
ax.set_title('Nullclines and Phase Flow')

# Legend
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], color='blue', linewidth=2, label='E-nullcline'),
    Line2D([0], [0], color='red', linewidth=2, label='I-nullcline'),
    Line2D([0], [0], color='black', linewidth=1, label='Trajectory')
]
ax.legend(handles=legend_elements)

plt.tight_layout()
plt.show()

### Step 6: Effect of Connection Weights

Explore how E/I balance affects dynamics:

In [ ]:
# Different wEE values (recurrent excitation)
wEE_values = [8.0, 10.0, 12.0, 14.0, 16.0]

fig, axes = plt.subplots(2, len(wEE_values), figsize=(15, 6))

for idx, wEE_val in enumerate(wEE_values):
    # Create model
    model = brainmass.WilsonCowanStep(1, wEE=wEE_val)
    brainstate.nn.init_all_states(model)
    
    def step(i):
        with brainstate.environ.context(i=i, t=i * brainstate.environ.get_dt()):
            model.update(rE_inp=0.5)
            return model.rE.value, model.rI.value
    
    rE, rI = brainstate.transform.for_loop(step, np.arange(10000))
    t = np.arange(10000) * brainstate.environ.get_dt()
    
    # Time series (top row)
    axes[0, idx].plot(t[-5000:], rE[-5000:, 0], 'b-', linewidth=0.8, label='E')
    axes[0, idx].plot(t[-5000:], rI[-5000:, 0], 'r-', linewidth=0.8, alpha=0.7, label='I')
    axes[0, idx].set_title(f'$w_{{EE}}$ = {wEE_val}')
    axes[0, idx].set_xlabel('Time (ms)')
    if idx == 0:
        axes[0, idx].set_ylabel('Activity')
        axes[0, idx].legend(fontsize=8)
    
    # Phase portrait (bottom row)
    axes[1, idx].plot(rE[-5000:, 0], rI[-5000:, 0], 'k-', linewidth=0.3, alpha=0.7)
    axes[1, idx].set_xlabel('E')
    if idx == 0:
        axes[1, idx].set_ylabel('I')

plt.suptitle('Effect of Recurrent Excitation ($w_{EE}$) on Dynamics', y=1.02, fontsize=14)
plt.tight_layout()
plt.show()

### Step 7: Whole-Brain Network

Build a network of Wilson-Cowan nodes coupled through structural connectivity:

In [ ]:
import os.path
import kagglehub

# Download HCP connectome data
path = kagglehub.dataset_download("oujago/hcp-gw-data-samples")
data = braintools.file.msgpack_load(os.path.join(path, "hcp-data-sample.msgpack"))

print(f"Connectome shape: {data['Cmat'].shape}")
print(f"Distance matrix shape: {data['Dmat'].shape}")

In [ ]:
class WilsonCowanNetwork(brainstate.nn.Module):
    """Whole-brain network of Wilson-Cowan nodes with delayed coupling."""
    
    def __init__(self, n_regions, conn_weight, delay_time, signal_speed=2.0, k=1.0):
        super().__init__()
        
        # Prepare connectivity
        conn = conn_weight.copy()
        np.fill_diagonal(conn, 0)  # no self-connections
        
        # Compute delays from distances
        delays = delay_time.copy() / signal_speed
        np.fill_diagonal(delays, 0)
        
        # Index array for delay buffer
        indices = np.tile(np.arange(n_regions)[None, :], (n_regions, 1))
        
        # Wilson-Cowan nodes with noise
        self.node = brainmass.WilsonCowanStep(
            n_regions,
            noise_E=brainmass.OUProcess(n_regions, sigma=0.01, tau=5.0 * u.ms),
            noise_I=brainmass.OUProcess(n_regions, sigma=0.01, tau=5.0 * u.ms),
        )
        
        # Diffusive coupling with delays
        self.coupling = brainmass.DiffusiveCoupling(
            self.node.prefetch_delay('rE', (delays * u.ms, indices),
                                     init=braintools.init.Uniform(0, 0.05)),
            self.node.prefetch('rE'),
            conn,
            k=k
        )
    
    def update(self):
        coupling_input = self.coupling()
        rE = self.node(rE_inp=coupling_input)
        return rE
    
    def step_run(self, i):
        with brainstate.environ.context(i=i, t=i * brainstate.environ.get_dt()):
            return self.update()

In [ ]:
# Create network
n_regions = data['Cmat'].shape[0]
net = WilsonCowanNetwork(
    n_regions,
    data['Cmat'],
    data['Dmat'],
    signal_speed=2.0,  # m/s
    k=0.5  # coupling strength
)
brainstate.nn.init_all_states(net)

# Run simulation (6 seconds)
dt_ms = float(brainstate.environ.get_dt() / u.ms)
n_steps = int(6000 / dt_ms)  # 6000 ms
indices = np.arange(n_steps)

print(f"Running {n_steps} steps ({n_steps * dt_ms / 1000:.1f} seconds)...")
rE_network = brainstate.transform.for_loop(net.step_run, indices)
print(f"Output shape: {rE_network.shape}")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Functional connectivity
fc = braintools.metric.functional_connectivity(rE_network[10000:, :])
im1 = axes[0].imshow(fc, cmap='RdBu_r', vmin=-1, vmax=1)
plt.colorbar(im1, ax=axes[0], label='Correlation')
axes[0].set_title('Functional Connectivity (FC)')
axes[0].set_xlabel('Region')
axes[0].set_ylabel('Region')

# Structural connectivity for comparison
sc = data['Cmat'].copy()
np.fill_diagonal(sc, 0)
im2 = axes[1].imshow(np.log10(sc + 1e-6), cmap='hot')
plt.colorbar(im2, ax=axes[1], label='log10(weight)')
axes[1].set_title('Structural Connectivity (SC)')
axes[1].set_xlabel('Region')
axes[1].set_ylabel('Region')

# Sample time series
t_network = indices * dt_ms
for i in range(0, n_regions, 10):
    axes[2].plot(t_network[10000:30000], rE_network[10000:30000, i] + i*0.1, 
                 linewidth=0.5, alpha=0.8)
axes[2].set_xlabel('Time (ms)')
axes[2].set_ylabel('rE (offset for visibility)')
axes[2].set_title('Regional Activity Time Series')

plt.tight_layout()
plt.show()

# Structure-function correlation
sc_flat = sc[np.triu_indices(n_regions, k=1)]
fc_flat = fc[np.triu_indices(n_regions, k=1)]
corr = np.corrcoef(sc_flat, fc_flat)[0, 1]
print(f"Structure-function correlation: {corr:.3f}")

## Exercises

### Exercise 1: E/I Balance Exploration

Modify the E-I coupling strength $w_{EI}$ and observe:
1. How does stronger inhibition affect oscillation frequency?
2. At what $w_{EI}$ do oscillations disappear?

```python
wEI_values = [10., 13., 16., 19., 22.]
# Hint: Plot oscillation amplitude vs wEI
```

### Exercise 2: Timescale Separation

Explore the effect of inhibitory time constant:
1. Set $\tau_I$ much slower than $\tau_E$ (e.g., 5x slower)
2. How does this affect the oscillation waveform?

### Exercise 3: Network Coupling Strength

Vary the global coupling strength `k` in the brain network:
1. What happens to FC at very low coupling?
2. What happens at very high coupling?
3. Is there an optimal coupling that maximizes structure-function correlation?

```python
k_values = [0.1, 0.5, 1.0, 2.0, 5.0]
sf_correlations = []
for k in k_values:
    # Build network, simulate, compute FC, measure SC-FC correlation
    pass
```

## Summary

In this tutorial, you learned:

1. **Wilson-Cowan equations**: E/I population dynamics with sigmoid transfer
2. **Bifurcation analysis**: Identifying oscillatory vs fixed-point regimes
3. **Nullcline analysis**: Understanding fixed points through phase plane
4. **Network dynamics**: Coupling multiple nodes with structural connectivity and delays

### Key Insights

- The Wilson-Cowan model captures **E/I balance** fundamental to cortical dynamics
- Oscillations emerge from the **interplay of excitation and inhibition**
- The bifurcation diagram reveals **parameter regimes** for different dynamics
- Network coupling with **realistic connectivity** produces structured FC

### Connection to Neural Phenomena

| Wilson-Cowan Feature | Neural Correlate |
|---------------------|------------------|
| E/I oscillations | Gamma rhythms |
| Bistability | Up/down states |
| Network synchronization | Large-scale coordination |

### Next Steps

- **Tutorial 021**: Jansen-Rit model for cortical columns and EEG rhythms
- **Tutorial 022**: Wong-Wang model for decision-making dynamics
- **Tutorial 040**: Coupling mechanisms in detail

## References

1. Wilson, H. R., & Cowan, J. D. (1972). Excitatory and inhibitory interactions in localized populations of model neurons. *Biophysical Journal*, 12(1), 1-24.

2. Wilson, H. R., & Cowan, J. D. (1973). A mathematical theory of the functional dynamics of cortical and thalamic nervous tissue. *Kybernetik*, 13(2), 55-80.

3. Destexhe, A., & Sejnowski, T. J. (2009). The Wilson-Cowan model, 36 years later. *Biological Cybernetics*, 101(1), 1-2.